In [26]:
import numpy as np
import cv2 as cv

FUNCTION TO CALCULATE DFT OF 2x2 matrix

In [27]:
#2x2 array is called mask
def dft_array(mask):
    """ Function to calculate dft of 2x2 matrix"""
    transform = np.zeros((2, 2, 3), dtype = 'float64')
    transform[0,0] = mask[0,0]/2 + mask[0,1]/2 + mask[1,0]/2 + mask[1,1]/2
    transform[0,1] = mask[0,0]/2 - mask[0,1]/2 + mask[1,0]/2 - mask[1,1]/2
    transform[1,0] = mask[0,0]/2 + mask[0,1]/2 - mask[1,0]/2 - mask[1,1]/2
    transform[1,1] = mask[0,0]/2 - mask[0,1]/2 - mask[1,0]/2 + mask[1,1]/2
    return transform

FUNCTION TO CALCULATE DFT OF IMAGE

In [28]:
def dft_image(image):
    """ Function to find the discrete fourier transform of an image by taking 2x2 array at a time""" 
    height = image.shape[0]
    width = image.shape[1]
    transformed_image = np.zeros((height, width, 3), dtype = 'float64')
    for i in range(0, height - 1, 2):
        for j in range(0, width - 1, 2):
            xcoordinates = np.array([(i, i),
                                     (i + 1, i + 1)])
            ycoordinates = np.array([(j, j + 1),
                                    (j, j + 1)])
            coordinates = (xcoordinates, ycoordinates)
            mask = image[coordinates]
            transformed_image[coordinates] = dft_array(mask)
    return transformed_image

FUNCTION TO HIDE MESSAGE IMAGE IN HOST IMAGE

In [29]:
def encode_image(host_image, message_image):
    """Function which encodes message image in the frequency domain representation of host image"""
    message_pixelarray = message_image.reshape(-1,3)
    height_host = host_image.shape[0]
    width_host = host_image.shape[1]
    host_pixelarray = host_image.reshape(-1,3)
    for i in range(message_pixelarray.shape[0]):
        host_pixelarray[i] += message_pixelarray[i]/10 
    newimage = host_pixelarray.reshape(height_host, width_host, 3)
    return newimage

FUNCTION TO DECODE MESSAGE IMAGE FROM HOST IMAGE

In [42]:
def decode_image(encoded_image, host_image, shape):
    """ Function which recovers message image from encoded image"""
    size = shape[0] * shape[1]
    message_image = cv.subtract(dft_image(encoded_image), dft_image(host_image))
    message_image = message_image.reshape(-1,3)
    message_image = message_image[range(size), :]
    message_image = message_image * 10
    message_image = message_image.reshape(shape[0], shape[1], 3)
    return message_image

In [43]:
def main():
    hostimage = input('Enter name of host image :')
    messageimage = input('Enter name of message image :')
    hostimage = cv.imread('sunset.jpg')
    hostimage = np.array(hostimage/255)
    messageimage = cv.imread('starry_night.jpg')
    messageimage = np.array(messageimage/255)
    cv.imshow('Host image', hostimage)
    cv.imshow('message image', messageimage)
    dft_of_host_image = dft_image(hostimage)
    encoded_frequency_domain_image = encode_image(dft_of_host_image, messageimage)
    encoded_spatial_domain_image = dft_image(encoded_frequency_domain_image)
    cv.imshow('encoded image', encoded_spatial_domain_image)
    decoded_message_image = decode_image(encoded_spatial_domain_image, hostimage, messageimage.shape)
    cv.imshow('decoded image', decoded_message_image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [44]:
main()

Enter name of host image :sunset.jpg
Enter name of message image :starry_night.jpg
